### Part 0. Google Colab Setup

Hopefully you're looking at this notebook in Colab!
1. First, make a copy of this notebook to your local drive, so you can edit it.
2. Go ahead and upload the OnionOrNot.csv file from  pp1.zip file.
3. Right click in the files panel, and select 'Create New Folder' - call this folder src
4. Upload all the files in the src/ folder from the pp1.zip file.

***NOTE: REMEMBER TO REGULARLY REDOWNLOAD ALL THE FILES IN SRC FROM COLAB.***

***IF YOU EDIT THE FILES IN COLAB, AND YOU DO NOT REDOWNLOAD THEM, YOU WILL LOSE YOUR WORK!***

If you want GPU's, you can always change your instance type to GPU directly in Colab.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/CIS5590/pp1/

/content/drive/MyDrive/CIS5590/pp1


In [ ]:
%ll

total 1961
-rw------- 1 root 1948772 Mar  9 17:58 OnionOrNot.csv
-rw------- 1 root   53887 Mar 18 23:29 pp1.ipynb
-rw------- 1 root      28 Mar  9 17:58 requirements.txt
drwx------ 3 root    4096 Mar 18 23:28 src/


### Part 1. Loading and Preprocessing Data
The following cell loads the OnionOrNot dataset, and tokenizes each data item

In [ ]:
# DO NOT MODIFY #
import torch
import random
import numpy as np
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
# this is how we select a GPU if it's avalible on your computer.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import pandas as pd
from src.preprocess import clean_text
import nltk
from tqdm import tqdm

nltk.download('punkt')
df = pd.read_csv("OnionOrNot.csv")
df["tokenized"] = df["text"].apply(lambda x: nltk.word_tokenize(clean_text(x.lower())))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Here's what the dataset looks like. You can index into specific rows with pandas, and try to guess some of these yourself :)

In [ ]:
df.head()

,text,label,tokenized
0,Entire Facebook Staff Laughs As Man Tightens P...,1,"[entire, facebook, staff, laughs, as, man, tig..."
1,Muslim Woman Denied Soda Can for Fear She Coul...,0,"[muslim, woman, denied, soda, can, for, fear, ..."
2,Bold Move: Hulu Has Announced That They’re Gon...,1,"[bold, move, :, hulu, has, announced, that, th..."
3,Despondent Jeff Bezos Realizes He’ll Have To W...,1,"[despondent, jeff, bezos, realizes, he, ’, ll,..."
4,"For men looking for great single women, online...",1,"[for, men, looking, for, great, single, women,..."


In [ ]:
df.iloc[42]

text         Customers continued to wait at drive-thru even...
label                                                        0
tokenized    [customers, continued, to, wait, at, drive-thr...
Name: 42, dtype: object

Now that we've loaded this dataset, we need to split the data into train, validation, and test sets. We also need to create a vocab map for words in our Onion dataset, which will map tokens to numbers. This will be useful later, since torch models can only use tensors of sequences of numbers as inputs. **Go to src/dataset.py, and fill out split_train_val_test, generate_vocab_map**

In [ ]:
## TODO: complete these methods in src/dataset.py
from src.dataset import split_train_val_test, generate_vocab_map

df = df.sample(frac=1)
train_df, val_df, test_df = split_train_val_test(df, props=[.8, .1, .1])
train_vocab, reverse_vocab = generate_vocab_map(train_df)

In [ ]:
# this line of code will help test your implementation
(len(train_df) / len(df)), (len(val_df) / len(df)), (len(test_df) / len(df))

(0.8, 0.1, 0.1)

PyTorch has custom Datset Classes that have very useful extentions. **Go to src/dataset.py, and fill out the HeadlineDataset class.** Refer to PyTorch documentation on Dataset Classes for help.

In [ ]:
df.head()

,text,label,tokenized
3111,Scientists Say Smelling Farts Might Prevent Ca...,0,"[scientists, say, smelling, farts, might, prev..."
18679,Cops Charge Man With “Destruction of Police Pr...,0,"[cops, charge, man, with, “, destruction, of, ..."
17472,Gandalf would kick Dumbledore’s ass in a fight...,0,"[gandalf, would, kick, dumbledore, ’, s, ass, ..."
21451,Woman Found She Was Farting Through Her Vagina...,0,"[woman, found, she, was, farting, through, her..."
20800,Japanese 'naked restaurant' to ban overweight ...,0,"[japanese, 'naked, restaurant, ', to, ban, ove..."


In [ ]:
from src.dataset import HeadlineDataset
from torch.utils.data import RandomSampler

train_dataset = HeadlineDataset(train_vocab, train_df)
val_dataset = HeadlineDataset(train_vocab, val_df)
test_dataset = HeadlineDataset(train_vocab, test_df)

# Now that we're wrapping our dataframes in PyTorch datsets, we can make use of PyTorch Random Samplers.
train_sampler = RandomSampler(train_dataset)
val_sampler = RandomSampler(val_dataset)
test_sampler = RandomSampler(test_dataset)

We can now use PyTorch DataLoaders to batch our data for us. **Go to src/dataset.py, and fill out collate_fn.** Refer to PyTorch documentation on Dataloaders for help.

In [ ]:
from torch.utils.data import DataLoader
from src.dataset import collate_fn
BATCH_SIZE = 16

train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=train_sampler, collate_fn=collate_fn)
val_iterator = DataLoader(val_dataset, batch_size=BATCH_SIZE, sampler=val_sampler, collate_fn=collate_fn)
test_iterator = DataLoader(test_dataset, batch_size=BATCH_SIZE, sampler=test_sampler, collate_fn=collate_fn)

In [ ]:
# # Use this to test your collate_fn implementation.
# # You can look at the shapes of x and y or put print
# # statements in collate_fn while running this snippet

for x, y in test_iterator:
    print(x, y)
    break
# test_iterator = DataLoader(test_dataset, batch_size=BATCH_SIZE, sampler=test_sampler, collate_fn=collate_fn)

tensor([[  24,    7,  101,    0, 7301,   20,    0, 2791,    0,    4,    0,   61,
           35,   10, 2847,    4,  247,    4, 7140,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [  86,    8,  136, 5901,  929,   42, 6894,  963,  408,  160,   15,  678,
           18,    0,  199,   81,    3,  870,    4,   94,    5,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 769,    8,  208,   10,    0,    8,   75,   36, 1904,   37,   18, 2803,
          648,    5,  769,    4,  114, 1121,  815,    2,  119,   10,  720,    3,
          222,    0,    0,    0,    0,    0,    0,    0],
        [ 131,  569,   27,    7, 9014,   36,  554,   53,  105,   10,  573, 1548,
           19,   10, 2648,    0, 2038,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [  84,  123, 2079,   47,    0,   23, 1109,    4,  114,    0,    2, 6583,
          284, 1208,  114,  143,  150, 

### Part 2: Modeling
Let's move to modeling, now that we have dataset iterators that batch our data for us. **Go to src/model.py, and follow the instructions in the file to create a basic neural network. Then, create your model using the class, and define hyperparameters.**

In [ ]:
from src.models import ClassificationModel
import torch.nn.functional as F

model = ClassificationModel(
    vocab_size=len(train_vocab)+1,
    embedding_dim=100,
    hidden_dim=128,
    output_dim=df.label.nunique()
).to('cuda')

### YOUR CODE GOES HERE (1 line of code) ###
### YOUR CODE ENDS HERE ###

In the following cell, **instantiate the model with some hyperparameters, and select an appropriate loss function and optimizer.**

Hint: we already use sigmoid in our model. What loss functions are availible for binary classification? Feel free to look at PyTorch docs for help!

In [ ]:
from torch.optim import AdamW

criterion, optimizer = torch.nn.NLLLoss(), AdamW(model.parameters(), lr=1e-3)
### YOUR CODE GOES HERE ###
### YOUR CODE ENDS HERE ###

### Part 3: Training and Evaluation
The final part of this HW involves training the model, and evaluating it at each epoch. **Fill out the train and test loops below.**

In [ ]:
# returns the total loss calculated from criterion
def train_loop(model, criterion, iterator):
    model.train()
    total_loss = 0
    for x, y in tqdm(iterator):
        ### YOUR CODE STARTS HERE (~6 lines of code) ###
        x,y = x.to('cuda'), y.to('cuda')
        outputs = model(x)
        loss = -criterion(outputs, y)
        optimizer.zero_grad()
        loss.backward()
        # torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item()
        ### YOUR CODE ENDS HERE ###
    return total_loss

# returns:
# - true: a Python boolean array of all the ground truth values
#         taken from the dataset iterator
# - pred: a Python boolean array of all model predictions.
def val_loop(model, criterion, iterator,batch_size:int=16):
    true, pred = torch.zeros(len(iterator.dataset)), torch.zeros(len(iterator.dataset))
    ### YOUR CODE STARTS HERE (~8 lines of code) ###
    with torch.no_grad():
        for n_val, (inputs, labels) in enumerate(tqdm(iterator)):
            start,stop = n_val*batch_size,(n_val+1)*batch_size
            true[start:stop] = labels
            inputs = inputs.to('cuda')
            outputs = model(inputs).argmax(dim=1)
            pred[start:stop] = outputs
    ### YOUR CODE ENDS HERE ###
    return true, pred

We also need evaluation metrics that tell us how well our model is doing on the validation set at each epoch. **Complete the functions in src/eval.py.**

In [ ]:
# To test your eval implementation, let's see how well the untrained model does on our dev dataset.
# It should do pretty poorly.
from src.eval_utils import binary_macro_f1, accuracy
true, pred = val_loop(model, criterion, val_iterator)
print(binary_macro_f1(true, pred))
print(accuracy(true, pred))

100%|██████████| 150/150 [00:00<00:00, 150.40it/s]


0.2683566438788089
0.3620833333333333


### Part 4: Actually training the model
Watch your model train :D You should be able to achieve a validation F-1 score of at least .8 if everything went correctly. **Feel free to adjust the number of epochs to prevent overfitting or underfitting.**

In [ ]:
TOTAL_EPOCHS = 10
for epoch in range(TOTAL_EPOCHS):
    train_loss = train_loop(model, criterion, train_iterator)
    true, pred = val_loop(model, criterion, val_iterator)
    print(f"EPOCH: {epoch}")
    print(f"TRAIN LOSS: {train_loss}")
    print(f"VAL F-1: {binary_macro_f1(true, pred)}")
    print(f"VAL ACC: {accuracy(true, pred)}")

100%|██████████| 150/150 [00:00<00:00, 647.05it/s]


EPOCH: 0
TRAIN LOSS: 2.622423866344434
VAL F-1: 0.4824438882051141
VAL ACC: 0.5841666666666666


100%|██████████| 150/150 [00:00<00:00, 622.44it/s]


EPOCH: 1
TRAIN LOSS: 0.017191540058547616
VAL F-1: 0.4547359109848926
VAL ACC: 0.5933333333333334


100%|██████████| 150/150 [00:00<00:00, 652.89it/s]


...

EPOCH: 7
TRAIN LOSS: 0.00027008422315510927
VAL F-1: 0.38618925831202044
VAL ACC: 0.6291666666666667


100%|██████████| 150/150 [00:00<00:00, 630.14it/s]


EPOCH: 8
TRAIN LOSS: 0.00014499580728610795
VAL F-1: 0.38618925831202044
VAL ACC: 0.6291666666666667


100%|██████████| 150/150 [00:00<00:00, 467.04it/s]


EPOCH: 9
TRAIN LOSS: 9.462735331533523e-05
VAL F-1: 0.38618925831202044
VAL ACC: 0.6291666666666667


In [ ]:
binary_macro_f1([0,0,1], [0,0,1])

1.0

In [ ]:
m = torch.nn.LogSoftmax(dim=1)
loss = torch.nn.NLLLoss()
# input is of size N x C = 3 x 5
input = torch.randn(3, 5, requires_grad=True)
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 4])
output = loss(m(input), target)

output

tensor(2.2992, grad_fn=<NllLossBackward0>)

In [ ]:
input

tensor([[ 0.4981, -0.9549, -0.2418, -1.1726,  0.3200],
        [-0.7306,  0.3876, -2.2699,  0.7869,  0.6669],
        [ 0.0847, -1.9157,  0.1480, -0.1058, -0.5251]], requires_grad=True)

We can also look at the models performance on the held-out test set, using the same val_loop we wrote earlier.

In [ ]:
true, pred = val_loop(model, criterion, test_iterator)
print(f"TEST F-1: {binary_macro_f1(true, pred)}")
print(f"TEST ACC: {accuracy(true, pred)}")

100%|██████████| 150/150 [00:00<00:00, 398.19it/s]


TEST F-1: 0.37662337662337664
TEST ACC: 0.6041666666666666


### Part 5: Analysis
Answer the following questions:
#### 1. What happens to the vocab size as you change the cutoff in the cell below? Can you explain this in the context of [Zipf's Law](https://en.wikipedia.org/wiki/Zipf%27s_law)?

In [ ]:
tmp_vocab, _ = generate_vocab_map(train_df, cutoff = 1)
len(tmp_vocab)

13298

#### 2. Can you describe what cases the model is getting wrong in the witheld test-set?

To do this, you'll need to create a new val_train_loop (``val_train_loop_incorrect``) so it returns incorrect sequences **and** you'll need to decode these sequences back into words.
Thankfully, you've already created a map that can convert encoded sequences back to regular English: you will find the ``reverse_vocab`` variable useful.

```
# i.e. using a reversed map of {"hi": 2, "hello": 3, "UNK": 0}
# we can turn [2, 3, 0] into this => ["hi", "hello", "UNK"]
```

In [ ]:
# Implement this however you like! It should look very similar to val_loop.
# Pass the test_iterator through this function to look at errors in the test set.
def val_train_loop_incorrect(
        model:torch.nn.Module,
        iterator:torch.utils.data.DataLoader,
        reverse_vocab:dict,
        batch_size:int=16
    )->None:
    true, pred = torch.zeros(len(iterator.dataset)), torch.zeros(len(iterator.dataset))
    with torch.no_grad():
        for n_val, (inputs, labels) in enumerate(tqdm(iterator)):
            # Generate a random permutation of column indices
            indices = torch.randperm(inputs.shape[1])
            # Apply this permutation to shuffle columns in the current row
            inputs = inputs[:, indices]

            for batch in inputs:
                print(' '.join([reverse_vocab[tk.item()] for tk in batch]))
                print()
            start,stop = n_val*batch_size,(n_val+1)*batch_size
            true[start:stop] = labels
            inputs = inputs.to('cuda')
            outputs = model(inputs).argmax(dim=1)
            pred[start:stop] = outputs
            # print(outputs)
    return true, pred

val_train_loop_incorrect(model, test_iterator, reverse_vocab)

  6%|▌         | 9/150 [00:00<00:01, 80.92it/s]

       trump     state runs pair eric  by through  screaming  dinner  naked        

   plate    has     much way more teacher  to too now fuck any on  yoga       students right

   up    never     would thought  ladder  a it    end  area        

   their    are     even still  clowns  makeup beautiful    without  these        

   right    man     for still    trump waiting  comment to the  mentally       him 

   to    to     $ be about   give paid in a speech   paul    nyc   income inequality

and t playing that find ’ work , home unemployment during ’ nintendo gamers ‘ news can in : ’ the next revealed animal bad their  ‘ your character crossing animal crossing

       child     are abuse his   '' allegations serial , claims ``  jimmy    killer peter  friend ,

   too    mcdonald     over ’  sues  many s   ’ ‘  russia        

       have     skills   hackers       '  is        

   from    over     make campaign lesbian grows   to  ' a elsa  backlash        

   who    is    free

 19%|█▊        | 28/150 [00:00<00:01, 88.09it/s]

    lawsuit settlement  dad teen 's  80,000   costs    facebook    $

an and him from mistaken a protect  a beautiful taken has : for  him home scientists story to  alien 

actual )   they hate  minority when christians  when a an hated    become  on minority (

look shit   and these  coast 8 years  spill the still gulf    after  birds like oil

hockey    $   store thief   steals canadian of sports    into  worth  ,

    '   not south pacific  exist island      sandy    to

legal screen    2  exclusive tim   of  s an    gives  ’  

, say   it to  porch man kept  used brain high beneath    stolen  get police ,

he  in  pass code  police he didn days iphone t so give  jail  ’  , got his

costs    daughter 's  help texas couple  for divorce care to    considering  health  pay

to on   says daughter  admission ben  a , about go college    jokes ' will yale scandal

    sans   email uber fired  comic people  in    15    using

and    spend it  a directors   ,   for    $  on  film

tree    p

 33%|███▎      | 49/150 [00:00<00:01, 93.74it/s]

...

 95%|█████████▍| 142/150 [00:01<00:00, 77.99it/s]

    court ’       bosses  ‘  iowa can  workers      fire :

 heroin  a of because       treatment  egypt  director center  of dies   overdose  in addiction

    becoming therapy       addicted  they  toddlers to   require     ipads so

’ to what seems disappointment but it least doing not in she this funny joined s devastating grandma the  facebook s so know ’ just :

it  until a  stuck .  are been days 300 's four owner tickets a captain .  are 's sold lift season and team

    ' applicants       test  indonesian   for   military     female virginity

    channel        of    weather         bias accused

 '   's revealed       media  ,  zuckerberg accounts  as password     hacked social

 deep '' in to just       whether  dead  court   `` or   meditation  is decide

 opening  pelosi approval following         %  trump to  nancy immediately   impeachment  98 rating

    from        vs.  an  working working   office     in home

    onion        america  news  the 's   source     fines

100%|██████████| 150/150 [00:01<00:00, 78.55it/s]

on court ,    '              supreme  cares  marriage :  gay    who

wants harry                   prince    ' banned      

to wants worth  mansion       million           $   minneapolis  sell    negative

opens                        dedicated to  museum    

 trillion last  work  the america    onion   finest  news     - -  lost to source hours 's   year

 : girlfriend  told  bring to    black         report people to  donald sterling  owner game   not

‘ : t  ’  that limits       who  could    santorum put bad  it wasn vote maybe on   ’

 daniel   loving  home     from         tearful station returning  family in      after

vote  making  bill  to gay    tell   exist      in kids illegal  russia passes  , people   it



(tensor([1., 1., 1.,  ..., 0., 1., 0.]),
 tensor([0., 0., 0.,  ..., 0., 0., 0.]))